<a href="https://colab.research.google.com/github/Ivan8Garcia/data-science/blob/main/Pandas_Tratamiento_Manipulacion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Problema del negocio**

se trata de un algoritmo capaz de analizar las caracteristicas  de una propiedad -como comodidades,tamaño, ocupacion del espacio en un  periodo determinado- y sugerir al anfitrion un precio a cobrar por tarifas diarias que garantice ganancias en momentos de alta demanda

##**1. Entendiendo el problema**

In [3]:
import pandas as pd

In [4]:
datos= pd.read_json("/content/datos_hosting.json")
datos.head()

FileNotFoundError: File /content/datos_hosting.json does not exist

In [ ]:
#usamos NORMALIZE para convertir el archivo JSON en un archivo normal,y nos lo devuelve siempre como un dataframe
#el primer "datos" es para el nuevo dataframe generado y el segundo "datos" es para especificar la primer variable que usamos para la ruta
# y se estará sobrescribiendo la primer variable datos porque ahora contendrá un data frame con todas las columnas del diccionario no abierto
datos= pd.json_normalize(datos["info_inmuebles"])
datos.head()

##**2. Datos Numericos**

In [ ]:
columnas= list(datos.columns)
columnas

In [ ]:
datos= datos.explode(columnas[3:])
datos

In [ ]:
#METODO PARA RESETEAR EL INDICE, ya que en la celda anterior nos aparecen muchas veces los mismos numeros en el indice
#primero se invoca al df datos, inplace es para actualizar en el mismo df  y drop es para borrar de la memoria el viejo indice
datos.reset_index(inplace= True, drop= True)
datos.head()

In [ ]:
#el tipo de datos OBJECT aparece cuando no sabe si es texto o numero; por lo tanto está mal y hay que seguir tratando los datos
datos.info()

In [ ]:
import numpy as np

In [ ]:
datos["max_hospedes"]=datos["max_hospedes"].astype(np.int64)


In [ ]:
datos.info()

In [ ]:
#haciendo una lista de las columnas a cambiar como tipo int64 y guardandola en una variable para hacerlo mas rapido
col_numericas= ["cantidad_baños","cantidad_cuartos","cantidad_camas"]

In [ ]:
datos[col_numericas]=datos[col_numericas].astype(np.int64)

In [ ]:
datos.info()

In [ ]:
datos["evaluacion_general"]=datos["evaluacion_general"].astype(np.float64)

In [ ]:
datos.info()

In [ ]:
datos["precio"]

In [ ]:
#tranformando la columna precio para que tengamos los valores adecuados en datos.info(), "aplicando un lamba x:"" con el metodo replace
#dentro del primer parentesis se sustituye el signo dolar por nada y en el segundo parentesis se sustituyen las posibles comas existentes por nada
# y el strip se usa para remover los espacios y solamente quede el numero
datos["precio"]=datos["precio"].apply(lambda x: x.replace("$","").replace(",","").strip())


In [ ]:
#vamos a la columna precio para verificar que se haya hecho el cambio de object a float y vemos que no pasó nadaporque no hemos hecho el astype
datos.info()

In [ ]:
#ahora si se aplica el cambio y se almacena en el df datos de la columna precio y lo confirmamos con un datos.info()
datos["precio"]= datos["precio"].astype(np.float64)

In [ ]:
datos.info()

In [ ]:
datos["precio"].head()

In [ ]:
#cuando es una sola columna se lee como serie y no es dataframe y por eso se usa .apply
datos[["cuota_deposito","cuota_limpieza"]].apply(lambda x: x.replace("$","").replace(",","").strip())

In [ ]:
#por ultimo cuando inspeccionamos mas de una columna se utiliza doble parentesis
#aqui se usa apply.map cuando son de dos columnas en adelante y porque tambien es un dataframe
#cuando es una sola columna se lee como serie y no es dataframe y por eso se usa .apply
datos[["cuota_deposito","cuota_limpieza"]]= datos[["cuota_deposito","cuota_limpieza"]].applymap(lambda x: x.replace("$","").replace(",","").strip())

In [ ]:
datos[["cuota_deposito","cuota_limpieza"]]= datos[["cuota_deposito","cuota_limpieza"]].astype(np.float64)

In [ ]:
datos.info()

##**3. Datos de Texto**

In [ ]:
datos.head()

In [ ]:
#Recordando que cuando es una sola columna tenemos una series no un df
datos["descripcion_local"].head()

In [ ]:
#aqui usaremos varios metodos, .str es para convertir a string y .lower es para quitar las mayusculas y tener todo en minusculas
datos["descripcion_local"].str.lower()

In [ ]:
datos["descripcion_local"]=datos["descripcion_local"].str.lower()

In [ ]:
datos.head()

In [ ]:
datos["descripcion_local"][3169]

In [ ]:
#
datos["descripcion_local"]=datos["descripcion_local"].str.replace('[^a-zA-Z0-9\-\']',' ', regex=True)

In [ ]:
datos["descripcion_local"]=datos["descripcion_local"].str.replace('(?<!\w)-(?<!\w)',' ', regex=True)

In [ ]:
datos["descripcion_local"].head()

CREANDO LA TOQUENIZACION (PALABRA A PALABRA)

In [ ]:
# una vez realizada la limpieza de todo tipo de caracteres innecesarios con REGEX, procedemos a la toquenizacion solamente usando .split()
datos["descripcion_local"]=datos["descripcion_local"].str.split()
datos.head()

In [ ]:
#si nos fijamos en la columna comodidades no se visualiza como una lista de tokens y la lista empieza con [], para ello hay que realizar el metodo .split
datos["comodidades"]=datos["comodidades"].str.replace('\{|}|\"','',regex=True)
datos.head()

In [ ]:
datos["comodidades"]=datos["comodidades"].str.split(",")
datos.head()

In [ ]:
# Transformamos el texto en letras minúsculas
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.lower()
# Substituímos los caracteres especiales
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.replace('[^a-zA-Z0-9\-\']', ' ', regex=True)
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.replace('(?<!\w)-(?!\w)', '', regex=True)
# Transformamos el texto en lista, formando el token
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.split()
datos.head()

#**4. DATOS DE TIEMPO**

In [37]:
import pandas as pd

In [38]:
dt_data= pd.read_json("/content/inmuebles_disponibles.json")

In [39]:
dt_data.head()

,id,fecha,lugar_disponible,precio
0,857,2016-01-04,False,None
1,857,2016-01-05,False,None
2,857,2016-01-06,False,None
3,857,2016-01-07,False,None
4,857,2016-01-08,False,None


In [40]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                365000 non-null  int64 
 1   fecha             365000 non-null  object
 2   lugar_disponible  365000 non-null  bool  
 3   precio            270547 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 11.5+ MB


In [41]:
dt_data["fecha"]=pd.to_datetime(dt_data["fecha"])

In [42]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                365000 non-null  int64         
 1   fecha             365000 non-null  datetime64[ns]
 2   lugar_disponible  365000 non-null  bool          
 3   precio            270547 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(1)
memory usage: 11.5+ MB


In [43]:
dt_data.head()

,id,fecha,lugar_disponible,precio
0,857,2016-01-04,False,None
1,857,2016-01-05,False,None
2,857,2016-01-06,False,None
3,857,2016-01-07,False,None
4,857,2016-01-08,False,None


In [44]:
#AGRUPACION POR MESES usando strftime y entre parentesis el formato que queremos obtener, en este solo queremos YEAR y MONTH
dt_data["fecha"].dt.strftime("%Y-%m")

,fecha
0,2016-01
1,2016-01
2,2016-01
3,2016-01
4,2016-01
...,...
364995,2016-12
364996,2016-12
364997,2016-12
364998,2017-01


In [45]:
#Agrupando por año y mes con el metodo .groupby  para saber con el metodo .sum cuantos lugares hay disponibles en cada mes
#.sum solo funciona si es booleano
#subset= subconjunto, asi se llama la nueva variable del df,antes llamado dt_data
#el resultado son los lugares disponibles en un determinado año y mes

subset=dt_data.groupby(dt_data["fecha"].dt.strftime("%Y-%m"))["lugar_disponible"].sum()
subset



,lugar_disponible
fecha,
2016-01,16543
2016-02,20128
2016-03,23357
2016-04,22597
2016-05,23842
2016-06,23651
2016-07,22329
2016-08,22529
2016-09,22471


In [46]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                365000 non-null  int64         
 1   fecha             365000 non-null  datetime64[ns]
 2   lugar_disponible  365000 non-null  bool          
 3   precio            270547 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(1)
memory usage: 11.5+ MB


In [27]:
dt_data.sample(8)

,id,fecha,lugar_disponible,precio
254842,3764,2016-03-16,False,None
340919,2223,2016-01-13,False,None
34237,385,2016-10-22,True,$85.00
201628,1934,2016-05-31,True,$260.00
70506,2204,2016-03-05,False,None
267400,40,2016-08-11,True,$125.00
204938,2376,2016-06-25,True,$86.00
299306,659,2016-01-10,False,None


In [47]:
#TRATANDO LA COLUMNA PRECIO

# utilizamos el método fillna para llenar los elementos vacíos por '0.0'
# definimos el parámetro de inplace para True para substituir en el DataFrame
dt_data['precio'].fillna('0.0', inplace = True)



<ipython-input-47-61e0597cd1b9>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dt_data['precio'].fillna('0.0', inplace = True)


In [48]:
# borramos el $ y las comas con apply lambda
dt_data['precio'] = dt_data['precio'].apply(lambda x: x.replace('$', '').replace(',',''))



In [50]:
# transformamos los tipos de datos para float64
dt_data['precio'] = dt_data['precio'].astype(np.float64)

# observamos el resultado final
dt_data

,id,fecha,lugar_disponible,precio
0,857,2016-01-04,False,0.0
1,857,2016-01-05,False,0.0
2,857,2016-01-06,False,0.0
3,857,2016-01-07,False,0.0
4,857,2016-01-08,False,0.0
...,...,...,...,...
364995,3279,2016-12-29,True,140.0
364996,3279,2016-12-30,True,140.0
364997,3279,2016-12-31,True,140.0
364998,3279,2017-01-01,True,140.0


In [51]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                365000 non-null  int64         
 1   fecha             365000 non-null  datetime64[ns]
 2   lugar_disponible  365000 non-null  bool          
 3   precio            365000 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1)
memory usage: 11.5 MB


#**DESAFIO 1**

Etapa 1

Proyecto Desafío 1: Ventas Online
El objetivo de este proyecto es analizar los resultados de un evento con los clientes de una empresa de venta online. Se recopiló un conjunto de datos que contiene los clientes que más gastaron en productos durante los 5 días de ventas, que es la duración del evento. Este análisis identificará al cliente con la mayor compra esta semana, quien recibirá un premio de la tienda, y posteriormente, puede ayudar a la empresa a crear nuevas estrategias para atraer más clientes.

La base de datos utilizada en este análisis es **dados_vendas_clientes.json** y contiene información importante sobre los clientes, como el nombre registrado del cliente, el monto total pagado al momento de la compra y el día de la compra.

Sabiendo esta información, el desafío del proyecto 1: ventas online será abrir la base de datos con Pandas y aplicar **json_normalize.**

In [ ]:
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_vendas_clientes.json')
# Aplicar json_normalize en la columna dados_vendas
datos = pd.json_normalize(datos['dados_vendas'])
# Mostrar valores
datos

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Así, el nuevo desafío del proyecto 1 será dividido en algunas metas:

**Eliminar datos en listas dentro del DataFrame;**

**Verificar tipos de datos;**

**Identificar columnas numéricas;**

**Transformar la columna numérica a tipo numérico.**

In [ ]:
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'Valor da compra'
datos['Valor da compra']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['Valor da compra'] = datos['Valor da compra'].apply(lambda x: x.replace('R$ ', '').replace(',','.').strip())
# Cambiar los tipo de datos para float
datos['Valor da compra'] = datos['Valor da compra'].astype(np.float64)
# Verificar la transformación
datos.info()

En el paso 2, trabajamos en la transformación de datos numéricos. Ahora podemos trabajar con valores textuales.

Debido a una inestabilidad en el sitio web de la empresa, tuvimos problemas con los nombres de los clientes durante el guardado. Esto resultó en una columna de nombres de clientes con una combinación de letras, mayúsculas y minúsculas, números y otros caracteres.

Sabiendo esto, manipula los textos de la columna **Cliente** para que el resultado sean los nombres de los clientes en letras minúsculas, con ausencia de caracteres especiales o números.

In [ ]:
# Transformar los textos de Cliente para texto en minúscula
datos['Cliente'] = datos['Cliente'].str.lower()
# Verificar el resultado
datos.head()

# Opción de substitución - necesario verificar el resultado de la substitución
# Regex no selecciona todas las letras de a-z y espacios en blanco ' '
# Todo que satisface el regex es borrado
datos['Cliente'].str.replace('[^a-z ]', '', regex=True)

# Realizar la substitución de los datos en la columna textual
datos['Cliente'] = datos['Cliente'].str.replace('[^a-z ]', '', regex=True).str.strip()
# Visualizar el resultado final
datos.head()

En los pasos anteriores ya hemos trabajado con varios tipos de datos, ahora podemos trabajar con datos de tiempo.

En la columna **Fecha de venta** tenemos fechas en el formato 'día/mes/año' **(dd/mm/AAAA)**. Transforme estos datos al tipo datetime y busque una forma de visualización de subconjunto que pueda contribuir al objetivo del contexto en el que se insertan los datos.

In [ ]:
# Transformar para el tipo datetime definiendo el formato de fecha como DD/MM/AAAA ('%d/%m/%Y')
datos['Data de venda'] = pd.to_datetime(datos['Data de venda'], format='%d/%m/%Y')
# Visualizar el resultado
datos

In [ ]:
# Verificar el resultado de la transformación
datos.info()

In [ ]:
# Calcular el total recaudado en compras por cada cliente
total_compras = datos.groupby(['Cliente'])['Valor da compra'].sum()
# Visualizar el resultado
total_compras

#**DESAFIO 2**

Administrar condominios es una tarea que requiere mucha atención y organización. Entre las diversas responsabilidades de gestión se encuentra el cobro del alquiler a los inquilinos. Para garantizar la buena salud financiera de la empresa, es fundamental que estos pagos se realicen de forma regular y puntual. Sin embargo, sabemos que esto no siempre sucede.

Teniendo esto en cuenta, propongo un desafío de procesamiento de datos con el objetivo de **analizar el retraso en el pago del alquiler en el condominio** de algunos residentes. Pongo a disposición la base de datos **dados_locacao_imoveis.json**, que contiene información sobre el departamento de los inquilinos, el día acordado para el pago del alquiler, el día en que se realiza el pago del alquiler y el monto del alquiler.

Con esta información, el desafío del proyecto 2: administración del condominio será similar al desafío del proyecto 1, abrir la base de datos con Pandas y aplicar **json_normalize** al DataFrame.

In [ ]:
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_locacao_imoveis.json')
# Aplicar json_normalize en la columna dados_locacao
datos = pd.json_normalize(datos['dados_locacao'])
# Mostrar valores
datos

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Entonces, de la misma manera que en el proyecto 1, el desafío del proyecto 2 está listado en algunas metas:

**Eliminar datos en listas dentro del DataFrame;**

**Verificar tipos de datos;**

**Identificar columnas numéricas;**

**Transformar la columna numérica a tipo numérico.**

In [ ]:
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'valor_aluguel'
datos['valor_aluguel']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['valor_aluguel'] = datos['valor_aluguel'].apply(lambda x: x.replace('$ ', '').replace(' reais', '').replace(',','.').strip())
# Cambiar los tipos de datos para float
datos['valor_aluguel'] = datos['valor_aluguel'].astype(np.float64)
# Verificar la transformación
datos.info()

En el paso 2, trabajamos en transformar los datos numéricos. Ahora podemos trabajar con valores textuales.

Buscando explicar la organización de la identificación de los apartamentos, durante la creación del conjunto de datos se añadió el texto **(blocoAP)**. Este texto informa que los nombres de los apartamentos están organizados con la letra mayúscula seguida del número del apartamento. Sin embargo, esto no aporta ninguna información a nuestros datos, por lo que resulta interesante eliminar este texto del conjunto de datos.

Con esto, manipule los textos de la columna apartamento para eliminar el texto **(blocoAP)** del DataFrame.

In [ ]:
# Opción de substitución - necesario verificar el resultado de la substitución
# Fue necesario adicionar la barra '\' para ser considerados los paréntesis como caracteres literales
datos['apartamento'].str.replace(' \(blocoAP\)', '')

# Realizar la substitución de los datos en la columna textual
datos['apartamento'] = datos['apartamento'].str.replace(' \(blocoAP\)', '')
# Visualizar el resultado final
datos

Al igual que en el Proyecto Desafío 1, trabajamos con todas las columnas excepto las que involucran fechas.

En las columnas **datas_de_pagamento** y **datas_combinadas_pagamento** tenemos fechas en el formato 'día/mes/año' (dd/mm/AAAA). Transforme estos datos al tipo datetime y busque una forma de visualización de subconjunto que pueda contribuir al objetivo del contexto en el que se insertan los datos.

In [ ]:
# Transformar para el tipo datetime definiendo el formato de fecha como DD/MM/AAAA ('%d/%m/%Y')
datos['datas_combinadas_pagamento'] = pd.to_datetime(datos['datas_combinadas_pagamento'], format='%d/%m/%Y')
datos['datas_de_pagamento'] = pd.to_datetime(datos['datas_de_pagamento'], format='%d/%m/%Y')

# Visualizar el resultado
datos



In [ ]:
# Para contribuir en la solución del contexto es posible calcular la diferencia de días
# entre la fecha combinada y la fecha de pagamento con dt.days
datos['atraso'] = (datos['datas_de_pagamento'] - datos['datas_combinadas_pagamento']).dt.days

# Observar el nuevo DataFrame
datos

In [ ]:
# Calcular el promedio de tiempo de atraso por apartamentos
media_atraso = datos.groupby(['apartamento'])['atraso'].mean()

# Visualizar el resultado
media_atraso

#**Ejemplo 1**

Para realizar un estudio financiero de distribución salarial en una empresa es necesario transformar los datos relativos a los cargos y prepararlos para un estudio estadístico.

Por lo tanto, al leer el archivo con los datos, te encuentras con el siguiente DataFrame:

Selecciona la alternativa que hace que los datos de las columnas cantidad y salario se conviertan en valores numéricos, es decir, columnas int y float sin existencia de texto que acompañe a cada valor.

Para ayudarlo con sus pruebas, intente usar el siguiente código para crear el DataFrame df y encontrar la solución para la actividad:

In [ ]:
import pandas as pd
import numpy as np

# datos de la empresa
datos = {
    'cargo': ['Gerencia', 'Coordinación', 'Supervisión', 'Analista', 'Asistente', 'Operación', 'Asistente', 'Prácticas', 'Asesoría', 'Consultoría'],
    'cantidad': ['2 personas', '1 persona', '3 personas', '4 personas', '5 personas', '3 personas', '2 personas', '1 persona', '1 persona', '1 persona'],
    'salario': ['$10.000 reales', '$8.000 reales', '$7.000 reales', '$5.000 reales', '$4.000 reales', '$3.500 reales', '$3.000 reales', '$1.500 reales', '$2.500 reales', '$6.000 reales']
}

# transformando el diccionario en DataFrame
df = pd.DataFrame(datos)

# df es el DataFrame con los datos de la empresa
df

In [ ]:
#Eliminamos los textos que están alrededor de los números con apply y lambda.
df['cantidad'] = df['cantidad'].apply(lambda x: x.replace('personas', ''))
df['salario'] = df['salario'].apply(lambda x: x.replace('$', '').replace('reales', ''))

#Luego, transformamos las columnas a tipo numérico con astype.
df['cantidad'] = df['cantidad'].astype(np.int64)
df['salario'] = df['salario'].astype(np.float64)